# <center> Numerical solution of PDEs
    
<center> Department of Physics, University of Surrey module: Energy, Entropy and Numerical Physics (PHY2063)

## Table of Contents:

* [Numerical Physics part of Energy, Entropy and Numerical Physics](#EENP-intro)
* [Introduction on Solving Partial Differential Equations](#pdes-intro)
* [Mathematics of diffusion in one dimension](#1d-diffusion)
* [Numerical solution of the diffusion PDE in one dimension](#1d-numerical-pdes)
* [Approximate formulas for the derivatives in one dimension](#1d-deriv-formula)
* [Task 1](#task-1)
* [Task 2](#task-2)

#### Learning Objectives:
> Revision on some basic concepts of PDEs

> Discussion of why PDEs are an important subject in numerical physics

> Look at specific example of diffusion equation in 1D and demonstrate the other mathematical equations that're required for us to solve it numerically



## 1) Numerical Physics part of Energy, Entropy and Numerical Physics <a class="anchor" id="EENP-intro"></a>

This numerical physics course is part of the second-year
Energy, Entropy and Numerical Physics (PHY2063) module,
and is online at the EENP module on SurreyLearn.
See there for assignments, deadlines etc.
The course is about numerically solving ODEs (ordinary differential equations) and
PDEs (partial differential equations), and introducing the
Monte Carlo technique and Bayesian statistics.

This part of the course is on PDEs.
PDEs are very common in physics. Maxwell's equations that govern electromagnetism
are PDEs, as is Schrödinger's equation in quantum mechanics.
Heat and particle diffusion obey PDEs, as do waves.

## 2) Introduction on Solving Partial Differential Equations <a class="anchor" id="pdes-intro"></a>

These notes aim to provide insight into how second-order partial
differential equations (PDEs) are solved on a computer.
We will only consider second-order PDEs as these are by far the most
common in physics.

In this part of the course we will consider an example of a time-dependent PDE, in
one dimension. We will consider the diffusion PDE as an example of a time-dependent PDE.
Diffusion is a very common phenomenon in physics. Electrons diffuse in metals and
semiconductors, molecules diffuse in liquids, etc., and thermal energy also moves
via diffusion in solids. 

PDEs are a very important topic to cover in numerical physics since as stated above they are a very common occurance in physics for describing physical systems. This means being able to accurate solve them is imperative to appropriately model some systems. In this notebook you will only learn a very basic method for solving a PDE however like for ODEs there are specialised python packages designed specifically for solving them. However these packages typically aren't intrinsic python packages and thus would require installation, additionally they're quite overkill for our purposes and so we won't be touching on them in this course, but if you are interested in learning about them for yourself you can find a wide range of resources available online. 

## 3) Mathematics of diffusion in one dimension <a class="anchor" id="1d-diffusion"></a>

In solids thermal energy typically moves via diffusion (in fluids convection also moves thermal energy around). If a solid is hot in one place
and colder in another, then diffusion is the process whereby thermal energy
moves from hot regions to cold regions, and so ultimately makes the temperature
uniform. As you have learnt or will learn in the EENP lectures, at equilibrium
the temperature is uniform.

So, to model the evolution in time and space of the temperature of a solid, $T$, we
need the diffusion PDE for the temperature $T$. This is

$$
\frac{\partial T({\bf r},t)}{\partial t}=D\nabla^2 T({\bf r},t)
$$

Here $D$ is the diffusion constant for the temperature.
$D$ has dimensions of length squared over time.
For diffusion in one dimension, along the $x$ axis,
this simplifies to

$$
\frac{\partial T(x,t)}{\partial t}=D\frac{\partial^2 T(x,t)}{\partial x^2}
$$

### 3.1) BCs for the diffusion PDE

To obtain a particular solution we need boundary conditions (BCs).
An example set of BCs is that
at $t=0$ the temperature is $20^{\circ}$C within
1cm of the origin and $0^{\circ}$C everywhere else along the $x$ axis.
We assume that
the system is effectively infinite along the $x$ axis, i.e., that
heat can diffuse out to $x\to\pm\infty$.
Then the BCs are

$$
T(x,t=0)=\left\{\begin{array}{cc}
0^{\circ}\mbox{C} &  x < -1~\mbox{cm}\\
20^{\circ}\mbox{C} & -1~\mbox{cm} \le x \le 1~\mbox{cm}\\
0^{\circ}\mbox{C} & x > 1~\mbox{cm}
\end{array}\right.
$$

These BCs plus the diffusion PDE allow us to calculate $T(x,t)$ at all times.

Note that these BCs are initial conditions, i.e., the function at $t=0$. They are what
we have if we know what the temperature profile is at one instant, and want to know
what it will look like in the future. These are very common BCs.

## 4) Numerical solution of the diffusion PDE in one dimension <a class="anchor" id="1d-numerical-pdes"></a>

Now that we know the mathematics we need to solve, we will look at how
we will numerically solve this PDE.

### 4.1) Numerical solution is in the form of an array

We will not determine the function $T(x)$ at all values of $x$,
but will again discretise space into points $h$ apart.
In one dimension this means that we are solving for a one-dimensional
array ${\tt temp[i]}$.
For small $h$ this will be a good approximation to the function. This array, i.e.,
its values, will depend on time $t$.

For example, if we want the temperature $T$ at $h=0.1$cm
intervals over a total length 80cm in the $x$ direction, and centred
at the origin, we need a grid of 801 $x$-values for ${\tt temp}$ the array of values of the temperature.

In [8]:
import numpy as np

# note the 0th index corresponds to -40cm as it is centered at the origin (index 400 corresponds to the origin 0cm)
temp = np.zeros(801)

Note that here we want to model a
system that is infinite along the $x$ axis, but of course we
cannot have an infinite array. So
we need to define the array to be big enough so that
the during the time interval we are interested in, the temperature diffusion is
confined to the region of the $x$ axis where we do have points.
As diffusion tends to spread out the profile,
an array of this size will be big enough only for diffusion over not too long a time.

### 4.2) Boundary conditions on the array

Here the BCs are the initial temperature distribution, i.e.,
the initial values of the array elements. We will apply the above BCs.
The first boundary condition we have already set when we built the array using the 
numpy zeros command, this set all elements of the array to zero. However we now need to 
set the elements between $-1$~cm and $+1$~cm to
$T=20^{\circ}$C, using:

In [13]:
for i in range(390,411,1):
    temp[i] = 20

which will work for $h=0.1$cm, as then there are 20 elements in the range -1cm to 1cm.


### Approximate formulas for the derivatives in one dimension <a class="anchor" id="1d-deriv-formula"></a>

The Taylor expansion expression for the value
of the function $T$ at the point $x+h$,
using a Taylor expansion about the point $x$, is

$$
T(x+h)=T(x)+ \left(\frac{\partial T(x)}{\partial x}\right)_xh+
\frac{1}{2}
\left(\frac{\partial^2 T(x)}{\partial x^2}\right)_{xx}h^2+
 \cdots
$$

where both derivatives are evaluated at the point $x$.
We denote the first derivative
of $T$ by $T_x(x)$, and the second
derivative by $T_{xx}(x)$.
Then

$$
T(x+h)=T(x)+ T_x(x)h + \frac{1}{2}T_{xx}(x)h^2 + \cdots
$$

The corresponding equation for the function $T$ at the point $x-h$ is

$$
T(x-h)=T(x)- T_x(x)h + \frac{1}{2}T_{xx}(x)h^2 + \cdots
$$

If we add the
equations for $T(x-h)$ and $T(x+h)$, we get

$$
T(x+h)+T(x-h)=2T(x) + T_{xx}(x)h^2 + \cdots
$$

We can easily rearrange this to give us an equation for the second
derivative, which is what we need. It is

$$
T_{xx}(x)=\frac{T(x+h)-2T(x)+T(x-h)}{h^2}
$$

In terms of the array elements, labelled $i$, this
becomes

$$
{\tt temp\_xx[i]}={\tt (temp[i+1]-2.0*temp[i]+temp[i-1]) / h**2} 
$$


### Task 1 <a class="anchor" id="task-1"></a>

In the box below plot the starting boundary conditions. What you should find is a central peak and zero everywhere else, this task shouldn't be difficult but it should help you visualise and conceptualise what is happening in our diffusion model. These boundary conditions decscribe our central point of heat before we effectively start allowing diffusion.

In [ ]:
# Plot the initial boundary conditions here

### 4.3) Numerical form of the Laplacian in one dimension

In one dimension the Laplacian of $T(x,t)$ is just

$$
\frac{\partial^2T}{\partial x^2}=T_{xx}(x,t)=\frac{T(x+h,t)-2T(x,t)+T(x-h,t)}{h^2}
$$

using our formula for derivatives in the above section.
In terms of the numbers of the array elements, $i$  this
becomes

$$
{\tt temp\_xx[i]}={\tt (temp[i+1]-2.0*temp[i]+temp[i-1]) / h**2} 
$$

where we call the temperature variable ${\tt temp}$ and the second
derivative of the temperature with respect to $x$, ${\tt temp\_xx}$. (Note that ${\tt \_}$   is an underscore
just jupyter notebook using that font displays it to look like a hyphen). 

### 4.4) Numerical form of the diffusion equation in one dimension

In one dimension the diffusion equation is

$$
\frac{\partial T(x,t)}{\partial t}=DT_{xx}(x,t)
$$

Here $D$ is the diffusion constant for temperature (dimensions of length squared
over time). Converting this to code is easy, it is just

$$
{\tt dtempdt[i]= diffc*temp\_xx[i]}
$$

or

$$
{\tt dtempdt[i]=diffc*(temp[i+1]-2.0*temp[i]+temp[i-1]) / h**2} 
$$

where ${\tt dtempdt}$ is the time derivative of the temperature, and ${\tt diffc}$
is the diffusion constant, $D$. This allows us to calculate
the values of the time derivative of the temperature at every point $i$.
Thus ${\tt dtempdt}$ will need to be defined as an array. One way to do this,
is using the same code as we used to define the ${\tt temp}$ array. (Although obviouosly the variable
needs to be named differently).

It is good to use numpy.zeros as it ensures you know what all the values are,
i.e., zero. If you do not do this and use numpy.empty then attempt
to use a variable or array element that is not been set by you, it
can contain any value, which will cause problems. The disadvantage being that it is a bit slower but 
it's unlikely this marginal speed cost will ever be significant in your code.

Once we have the time derivative in ${\tt dtempdt}$, we can integrate
the PDE forward in time using the Euler method (as we used for ODEs) at each point

$$
{\tt temp[i]=temp[i]+dt*dtempdt[i]}
$$

for ${\tt dt}$ a small time step. Note that for PDEs we need a small step size
along the $x$ axis, ${\tt h}$, and a small step size along the time axis, ${\tt dt}$.

### 4.5) Structure of the program

The structure of the program is simple. We need an outer do loop over time that
integrates forward in time from the start at $t=0$, up until the end time
$t_{end}$, in $n_t$ steps of (small) size $\delta t$. Nested inside this loop
we have two do loops over $x$, one loop follows after the other. 
The first one calculates the values of $\partial T/\partial t$
at each position (i.e., the array ${\tt dtempdt}$),
and then the second do loop uses these values in the Euler method to calculate
the temperature at all $x$ values, at the next time step.

### 4.6) Step sizes and errors

As with ODEs, our solution is approximate. Here the size of the error depends on
both the step size along the $x$ axis, $h$, and the step size along the time axies, $\delta t$. The smaller is $h$ (i.e.,
the finer is the grid) the smaller the error due to only working with a finite set of points along the $x$ axis. But here as we are integrating with
time there is also an error due to the time integration. To keep the error due to the time integration small we can only make small changes
in the ${\tt temp[i]}$ elements at each step.
These changes are proportional to $\delta t$, and to $D$ but also scale as $1/h^2$ (from the $h$
dependence in the Laplacian). These numbers combine to form a dimensionless
group $(D\delta t)/h^2$. This group must be much less than one, to keep 
the errors in the time integration low. If this number is too large, the time integration
can even fail, which can cause the program to crash. It is a good idea to write
this number to the screen so you can check its value.

### Task 2 <a class="anchor" id="task-2"></a>

The method that we are using to solve the diffusion equation is called the forwards difference method (FDM) and it is well documented online and in literature. The task is to look at the following resources to gain a better appreciation and understanding of the method, this will be very useful to you as it is also the method you will employ for your solution in the assessment.

<br>  
The first resource is the wikipedia article on FDM:

https://en.wikipedia.org/wiki/Finite_difference_method
    
<br>    
The second resource is the following online book:

http://www.cs.man.ac.uk/~fumie/tmp/introductory-finite-difference-methods-for-pdes.pdf
    
In this book you should specifically look at chapters 1 and 2 as a general introduction to PDEs, then specifically chapter 5.2 for the diffusion equation that we are covering.

<br>  
Additionally for the analytical solution of the diffusion equation (very useful for checking if your assessment code is correct) you can see the notes here:

http://personal.ph.surrey.ac.uk/~phs1rs/teaching/l3_pdes.pdf
    
All of these notes are relevant and helpful but particularly the Gaussian solution. 